In [3]:
import librosa
import IPython.display as ipd

y, sr = librosa.load(r'/workspaces/Librosa-Sound-Organiser-/data/glass.wav')
ipd.Audio(y, rate=sr)

/tmp/ipykernel_8499/1241138670.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(r'/workspaces/Librosa-Sound-Organiser-/data/glass.wav')


EOFError: 